# Execution environment

In [ ]:
import random
import math

### Fish

In [8]:
# Submission

def fish(a,b,c,d,e,f,g,h,*args,**kwargs):
    cc = lambda x:(100*e**2)
    
    return [harvest, feed, fry, expand], [a, b], {'d': d, 'e': e}


# Training


### Test

In [ ]:
# Init

turn = 0 #a
credit = 100 #b
population = 0 #c
food = 0 #d
pool_level = 1 #e
food_price = random.uniform() #f
fry_price = 0 #g
expand_price = 0 #h
